In [3]:
#librerie necessarie
%tensorflow_version 2.x 
import tensorflow as tf

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
!pip install tensorflow_probability #==0.8.0rc0 --user --upgrade
import tensorflow_probability as tfp  # We are using a different module from tensorflow this time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [6]:
#per importare dataset dal proprio Drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [7]:
# Import del dataset
dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/prec_TOS19000077.csv', delimiter=";") # dataset completo
pd.DataFrame.head(dataset)

,gg/mm/aaaa,Precipitazione [mm],Tipo Dato
0,14/12/2016,NaN,@
1,15/12/2016,NaN,@
2,16/12/2016,NaN,@
3,17/12/2016,NaN,@
4,18/12/2016,NaN,@


In [8]:
#voglio solo i dati da Giugno 2022 a fine Luglio 2022
#prima rinomino le colonne del .csv per evitare problemi di naming dati dagli spazi
dataset.rename(columns={dataset.columns[0]:'date'}, inplace=True)
dataset.rename(columns={dataset.columns[1]:'precipitazione'}, inplace=True)
dataset.rename(columns={dataset.columns[2]:'tipo_dato'}, inplace=True)
dataset.columns

Index(['date', 'precipitazione', 'tipo_dato'], dtype='object')

In [9]:
df = pd.DataFrame

In [10]:
#converto la colonna data in formato data
dataset.info
dataset["date"]= pd.to_datetime(dataset["date"])

In [38]:
#filtro i record tra giugno e luglio
dataset = dataset[dataset['date'].between('2022/06/01', '2022/07/31')]
df.describe(dataset)
np.unique(dataset['precipitazione'])
#replace delle , coi . perché davano fastidio nei calcoli, poi converto la colonna in numero
dataset['precipitazione'] = dataset['precipitazione'].str.replace(',','.')
pd.unique(dataset['precipitazione'])
dataset['precipitazione'] = pd.to_numeric(dataset['precipitazione'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


In [53]:
#creo una nuova colonna, ad ogni riga assegno 1 se ha piovuto, 0 se non ha piovuto
bool_pioggia = []
for i in dataset['precipitazione']:
  if i > 0:
    bool_pioggia.append(1)
  else:
    bool_pioggia.append(0)
dataset['bool_pioggia'] = bool_pioggia

In [95]:
#probabilità iniziali
pd.DataFrame.count(dataset) #51 valori nel dataset
piovoso = sum(dataset['bool_pioggia'])/51 # percentuale che si inizi con un giorno di pioggia, 0.14
solare = 1 - sum(dataset['bool_pioggia'])/51 #percentuale che si inizi con un giorno di sole, 0.86

In [71]:
#funzione per definire la matrice stocastica di transizione degli stati
def transition_matrix(transitions):
    n = 1+ max(transitions) #numero di stati

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #per convertire in probabilità:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

In [73]:
#la matrice stocastica per i nostri dati
m = transition_matrix(dataset['bool_pioggia'])
for row in m: print(' '.join('{0:.2f}'.format(x) for x in row))

#   0     1
#0 0.91  0.09
#1 0.67  0.33

0.91 0.09
0.67 0.33


In [113]:
#modello di Markov
tfd = tfp.distributions 
initial_distribution = tfd.Categorical(probs=[0.14, 0.86])  # probabilità iniziali
transition_distribution = tfd.Categorical(probs=[[0.91, 0.09],
                                                 [0.67, 0.33]])  # probabilità di transizione degli stati (matrice stocastica)
observation_distribution = tfd.Normal(loc=[20., 35.], scale=[5., 5.])  #calcoli sulle temperature
#purtroppo il sensore non le aveva, le ho improvvisate googlando un poco, neanche sensori vicini avevano
#le temperature per i giorni presi in esame

# in loc=[0,1] va inserita la media della temperatura per i due diversi stati
# in scale =[0,1] la deviazione standard della temperatura per i due diversi stati

In [114]:
#modello
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=7) # numero di giorni da prevedere

In [115]:
mean = model.mean()

#per produrre i calcoli dal grafo che tf ha creato e vedere le previsioni
with tf.compat.v1.Session() as sess:  
  print(mean.numpy())

[32.9      24.446001 22.417042 21.93009  21.813221 21.785172 21.778439]
